In [1]:
import os,sys
import numpy as np
import xarray as xr
import zarr
# from dask.distributed import Client
import multiprocessing.popen_spawn_posix
from distributed import Client

c = Client()
c

Client Scheduler: tcp://127.0.0.1:45723 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 5 Cores: 10 Memory: 322.12 GB


# STERIC

In [2]:
%%time

prefix = "eORCA025.L75-IMHOTEP"
chunk_size = {"x":133,"y":320}

nexp="EGAI"
diro = "/gpfsscratch/rech/cli/uor98hu/ETUDE3/ENS_VAR_BUDGET/"+nexp+"/STERIC/"
ii=1
totb=[]

for Emb in ['001','002','003','004','005','006','007','008','009','010']:
    diridatref="/gpfsscratch/rech/cli/uor98hu/ETUDE3/ENS_VAR_BUDGET/STERIC/"+nexp+"/"+Emb+"/"
    data = xr.open_mfdataset(diridatref+"????/"+prefix+'.'+nexp+'.'+Emb+'_y*.1m_steric_0_bottom.nc', chunks = chunk_size, parallel=True).sosteric.squeeze()
    totb.append(data.assign_coords({'ens':ii}))
    ii+=1
    
sosteric= xr.concat(totb, 'ens') # full time period

month_length = sosteric.time_counter.dt.days_in_month
# Calculate the weights by grouping by 'time_counter.year'.
weights =  month_length.groupby("time_counter.year") / month_length.groupby("time_counter.year").sum() 
# Calculate the weighted average
sosteric_yearly = (sosteric * weights).groupby("time_counter.year").sum("time_counter")

sosteric_yearly.to_dataset(name="sosteric").to_zarr(diro+nexp+"1y_steric_0_bottom_1980_2018.zarr", mode='w')

distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 33% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

CPU times: user 21min 25s, sys: 26.9 s, total: 21min 52s
Wall time: 22min 41s


# HALO et THERMO

In [2]:
prefix = "eORCA025.L75-IMHOTEP"
chunk_size = {"x":133,"y":320}

for nexp in ["ES"]: # could be "EAI", 'EGAI'
    print(nexp)
    for COMP in ['HALO', 'THERMO']: # could be "STERIC", "THERMO", "HALO"
        print(COMP)    
        diro = "/gpfsscratch/rech/cli/uor98hu/ETUDE3/ENS_VAR_BUDGET/"+nexp+"/"+COMP+"/"
        ii=1
        totb=[]
                
        for Emb in ['001','002','003','004','005','006','007','008','009','010']:
            print(Emb)
            diridatref="/gpfsscratch/rech/cli/uor98hu/ETUDE3/ENS_VAR_BUDGET/"+COMP+"/"+nexp+"/"+Emb+"/"
            data = xr.open_mfdataset(diridatref+"????/"+prefix+'.'+nexp+'.'+Emb+'_y*.1m_steric_0_bottom.nc', chunks = chunk_size, parallel=True).sosteric.squeeze()
        
            totb.append(data.assign_coords({'ens':ii}))
            ii+=1
        
        sosteric= xr.concat(totb, 'ens') # full time period
        
        month_length = sosteric.time_counter.dt.days_in_month
        # Calculate the weights by grouping by 'time_counter.year'.
        weights =  month_length.groupby("time_counter.year") / month_length.groupby("time_counter.year").sum() 
        # Calculate the weighted average
        sosteric_yearly = (sosteric * weights).groupby("time_counter.year").sum("time_counter")
        
        sosteric_yearly = sosteric_yearly.compute()
        
        sosteric_yearly.to_dataset(name="sosteric").to_zarr(diro+nexp+"1y_steric_0_bottom_1980_2018.zarr", mode='w')

ES
HALO
001
002
003
004
005
006
007
008
009
010


distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 34% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 35% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took

THERMO
001


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


002


...

distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


005


distributed.utils_perf - WARNING - full garbage collections took 13% CPU time recently (threshold: 10%)


006
007
008
009
010


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)


# all depths ranges

In [2]:
prefix = "eORCA025.L75-IMHOTEP"
chunk_size = {"x":133,"y":320}
COMP = "HALO" # could be "STERIC" ...
for nexp in ["EGAI", "EAI", "ES"]:

    for dep in ["50", "100", "300", "700", "2000"]: 
            
        diro = "/gpfsscratch/rech/cli/uor98hu/ETUDE3/ENS_VAR_BUDGET/"+nexp+"/"+COMP+"/"
        ii=1
        totb=[]
        
        for Emb in ['001','002','003','004','005','006','007','008','009','010']:
            diridatref="/gpfsscratch/rech/cli/uor98hu/ETUDE3/ENS_VAR_BUDGET/"+COMP+"/"+nexp+"/"+Emb+"/"
            data = xr.open_mfdataset(diridatref+"????/"+prefix+'.'+nexp+'.'+Emb+"_y*.1m_steric_0_"+dep+"m.nc", parallel=True).sosteric.squeeze()
            totb.append(data.assign_coords({'ens':ii}))
            ii+=1
            
        sosteric= xr.concat(totb, 'ens') # full time period
        
        month_length = sosteric.time_counter.dt.days_in_month
        # Calculate the weights by grouping by 'time_counter.year'.
        weights =  month_length.groupby("time_counter.year") / month_length.groupby("time_counter.year").sum() 


        # Calculate the weighted average
        sosteric_yearly = (sosteric * weights).groupby("time_counter.year").sum("time_counter")
        # save avec le bon chunking
        sosteric_yearly.chunk(chunk_size).to_dataset(name="sosteric").to_zarr(diro+nexp+"1y_steric_0_"+dep+"m_1980_2018.zarr", mode='w')


distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 12% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 10% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took 11% CPU time recently (threshold: 10%)
distributed.utils_perf - WARNING - full garbage collections took